In [5]:
import pandas as pd
import numpy as np
from glob import glob


In [6]:

file_lst = glob("raw/*.csv")


In [7]:
class RES:
    def __init__(self, features=[]):
        self.ft_n = len(features)
        self.features=features
        self.RES = np.empty((1, self.ft_n))
        self.RES_len=0
        self.RES_cap=1

    def add(self,x):
        if isinstance(x, pd.DataFrame):
            x = x.to_numpy()        
        size = x.shape[0]
        if (size + self.RES_len > self.RES_cap):
            while (size + self.RES_len > self.RES_cap):
                self.RES_cap *= 2
            
            newRes = np.empty((self.RES_cap, self.ft_n))
            newRes[0:self.RES_len,:] = self.RES[0:self.RES_len,:]
            self.RES = newRes
        self.RES[self.RES_len:self.RES_len+size, :] = x
        self.RES_len += size

    def getDf(self):
        return pd.DataFrame(self.RES[0:self.RES_len,:], columns=self.features)

    def __len__(self):
        return self.RES_len
        
    def __str__(self):
        return self.RES.__str__()


In [8]:

n_normal = 0
n_attack = 0
req_ft=['duration','src_bytes','dst_bytes','missed_bytes','src_pkts','dst_pkts','label']
print(len(req_ft))
r = RES(req_ft)
for f in file_lst:    
    df = pd.read_csv(f, low_memory=False)  
    df = df[df['type'].isin(['normal', 'dos', 'ddos', 'scanning'])]
    df = df.loc[:, df.columns.isin(req_ft)]
    size = df.shape[0]
    
    at_df = df[df['label'] == 1]
    nr_df = df[df['label'] == 0]
    n_nr = nr_df.shape[0]
    n_normal += n_nr
    n_attack += at_df.shape[0]    
    print(len(r))    
    
    rng = np.random.default_rng()
    r.add(rng.choice(df.to_numpy(), size=max(0,int(size*0.03)), replace=False))
    del df
    


7
0
30000
60000
90000
90432
116676
146676
147308
177308
207308
227484
257484
287484
317484
347484
377484
407484
414533
432919
462919
492919
493066


In [9]:
print (f"NORMAL: {n_normal} ATTACK: {n_attack}")

NORMAL: 755124 ATTACK: 16680497


In [10]:
df = r.getDf()
df.to_csv("processed/imbalanced_dataset.csv", index=False)

In [11]:


df = pd.read_csv("processed/imbalanced_dataset.csv")
print(df[df.isnull().any(axis=1)].shape[0])
N=df.shape[0]
train_size=int((N*3)/4)
print(f"N: {N}, TS:{train_size}")
test_df = df.loc[train_size:N]
df = df.loc[0:train_size]

0
N: 523066, TS:392299


In [12]:
test_df.to_csv('processed/test_data.csv', index=False)

In [13]:
# from imblearn.over_sampling import ADASYN
# Y = df['label'].to_numpy().astype(np.uint8)
# X = df.drop('label', axis=1).to_numpy()
# df[df['label'].isnull()].head()
# print(Y[0:100])



In [14]:
# from imblearn.over_sampling import ADASYN
# X_r, Y_r = ADASYN().fit_resample(X, Y)
# res=np.empty((X_r.shape[0], df.shape[1]))
# res[:, 0:-1] = X_r
# res[:, -1] = Y_r
# pd.DataFrame(res, columns = df.columns).to_csv("processed/balanced_dataset.csv", index=False)

In [15]:
# np.bincount(Y_r)

In [16]:
# np.random.shuffle(res)


In [17]:
df.to_csv("processed/train_data.csv", index=False)

In [18]:
df.tail(15)

,duration,src_bytes,dst_bytes,missed_bytes,src_pkts,dst_pkts,label
392285,61.066910,0.0,0.0,0.0,3.0,2.0,1.0
392286,0.011342,212.0,107.0,0.0,6.0,4.0,1.0
392287,10.412138,0.0,0.0,0.0,4.0,3.0,1.0
392288,61.675492,0.0,0.0,0.0,3.0,2.0,1.0
392289,61.096238,0.0,0.0,0.0,3.0,2.0,1.0
392290,0.000082,0.0,0.0,0.0,2.0,1.0,1.0
392291,0.007699,208.0,107.0,0.0,6.0,4.0,1.0
392292,63.475032,0.0,0.0,0.0,3.0,2.0,1.0
392293,9.904154,0.0,0.0,0.0,4.0,3.0,1.0
392294,60.408946,0.0,0.0,0.0,3.0,2.0,1.0
